In [1]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import os
from nltk.corpus import stopwords
import operator
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [2]:
stop_words = set(stopwords.words("english"))
l = ['Xref', 'os','ch','AX','would','Go','prove','msdos','like','article','think','sfu', 'LK','use', 'ms', 'References' ,'alt', 'Newsgroups',
     'ca', 'sFvE', 'Aq', 'murdoch', 'acc','jose','ie','umich','myers','louis','mf','al','fnal','xterm','document','ba',
     'anywhere','tx','yet','tu','su','ps','wa','many','hours','get','usl','DY', 'tm', 'dsinc', 'tw', 'UT', 'trw',
     'anymore', 'decent','Mb','MP','MO', 'MG', 'latter','ss'
     '','dt','wayne','uug','fsu', 'cantaloupe','ksu','sunday','statement', 'srv', 'APR', 'VM', 'MCGILL', 'CA','Date', 'Fri', 'Apr', 'GMT','Sat','Mon','Tue','Wed',
     'Thu','least','ufl','vt', 'hyess','[pp','As','[', 'In','APR','net','uunet','HST','But','certainly','Ben','lines',
     'cs', 'cmu', 'Nyx','edu', 'comp', 'windows','Reply', 'To', 'misc','Lines','From', 'slc', 'cim', 'cdc', 
     'com', 'Steve', 'Chesney', 'x','I','Subject', 'G', 'qxF', 'fekVH','Sender', 'news','along','decwrl','mu',
     'fc', 'hp', 'news', 'daemon','Nntp', 'Posting', 'Host', 'hpfcmrc', 'fc', 'hp','Message','aa','figure','not',
     'ID','ux','nz', 'sL', 'fc', 'hp','Organization', 'Hewlett','b', 'Packard', 'Fort','MR','dr','ti','md'
     'Collins', 'CO','Newsreader','MA', 'Bz', 'raistlin','nt', 'udev', 'TIN', '[version', 'PL','Path',
     'crabapple', 'bb', 'andrew','cc', 'caen','Woah','fwi', 'uva', 'nl','u', 'k', 'v', 'w', 'b', 'j', 'i',
     't', 'z', 'g', 'c', 'y', 'o', 'h', 'r', 's', 'f', 'q', 'l', 'd', 'a', 'n', 'p', 'e', 'm', 'x',
     'news', 'sei', 'cis', 'ohio', 'state', 'zaphod','Could', 'sdd', 'hp','Hi','oh','di','ok','summary'
     'col', 'hp', 'fc', 'hp', 'rodc', 'Cerkoney','CHECK', 'MENAHEM', 'BEGIN', 'DAIRIES','psuvm',
    'joe','joseph','fr','none','uh',]
l = [l[i].lower() for i in range(len(l))]
stop_words.update(l)

In [3]:
spec_chars= set([ '1' ,'2' ,'3' ,'4','5','6','7','8','9','0','\n','\t','<','>','!','@',
             "#","$","%","^","&","*",'~',"(",")","?","/","//",":","+",";", ']',"`"
             '=' , "." , "," , "-" , "'" , '"' , 
             "_","|", "\\" ,"]" ])

In [4]:
def removeSpecialChars(l):
    for i in spec_chars:
        l=l.replace(i," ")
    return l

In [5]:
def getFilteredSentences(f):
    lines = f.readlines()
    sentences = []
    for l in lines:
        l1 = removeSpecialChars(l)
        words = l1.split(' ')
        sentences.append([w for w in words if w.lower() not in stop_words and w.isalpha()])
        if [] in sentences:
            sentences.remove([])
    return sentences

In [6]:
def updateVocab(sentences , vocab):
    for s in sentences:
        for word in s:
            if word in vocab:
                vocab[word]=vocab[word]+1
            else:
                vocab[word]=1
    return

In [7]:
def build_vocab(dictionary , k):
    vocab = {}
    path = "/home/addrija/ML/naive bayes/20_newsgroups"
    for directory in dictionary:
        all_files = dictionary[directory]
        for f in all_files:
            f_path = path +'/' + directory + '/' +f
            f_opened = open(f_path,'r',encoding ='ISO-8859-1')
            sentences = getFilteredSentences(f_opened)
            updateVocab(sentences , vocab)
    v_sorted = sorted(vocab.items() , key = operator.itemgetter(1) , reverse=True)  
    vocab = [v_sorted[i][0] for i in range(k)]    
    return vocab

In [9]:
def organiseData(dictionary , vocab , path ):
    v_size = len(vocab)
    y_values = list(dictionary.keys())
    y = []
    x = []
    for directory in dictionary:
        all_files = dictionary[directory]
        for f in all_files:
            f_path = path + '/' + directory + '/' + f
            f_opened = open(f_path,'r',encoding ='ISO-8859-1')
            sentences = getFilteredSentences(f_opened)
            curr_list = np.zeros(v_size , dtype = int)
            for s in sentences:
                for w in s:
                    if w in vocab:
                        i = vocab.index(w)
                        curr_list[i] = curr_list[i]+1
            x.append(curr_list)
            y.append(directory)
    x = np.array(x)
    y = np.array(y)
    return x,y

In [10]:
def getDataFromDir(path):
    d = []
    dictionary = {}
    f =[]
    for (root,dirs,files) in os.walk(path , topdown = True):
        d.append(dirs)
        break 
    d = np.reshape(d,(len(d[0],)))
    for directory in d:
        curr_path = path+'/'+directory 
        files_in_curr_dir = os.listdir(curr_path)
        dictionary[directory]=files_in_curr_dir
    return dictionary

In [12]:
path = '/home/addrija/ML/naive bayes/20_newsgroups'
dictionary_train = getDataFromDir(path)
k = 2500
vocab = build_vocab(dictionary_train ,k)
x_train , y_train = organiseData(dictionary_train , vocab , path)

path = '/home/addrija/ML/naive bayes/mini_newsgroups'
dictionary_test = getDataFromDir(path)
x_test , y_test = organiseData(dictionary_test , vocab , path)

clf = MultinomialNB()
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [13]:
from sklearn.metrics import classification_report
print(classification_report(y_test , y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.78      0.84      0.81       100
           comp.graphics       0.81      0.88      0.84       100
 comp.os.ms-windows.misc       0.94      0.59      0.72       100
comp.sys.ibm.pc.hardware       0.80      0.90      0.85       100
   comp.sys.mac.hardware       0.85      0.93      0.89       100
          comp.windows.x       0.88      0.85      0.86       100
            misc.forsale       0.88      0.98      0.92       100
               rec.autos       0.94      0.95      0.95       100
         rec.motorcycles       0.95      0.96      0.96       100
      rec.sport.baseball       0.97      0.97      0.97       100
        rec.sport.hockey       0.99      0.94      0.96       100
               sci.crypt       0.97      0.96      0.96       100
         sci.electronics       0.89      0.94      0.91       100
                 sci.med       0.96      0.94      0.95       100
         